In [1]:
# Set up
import torch
import torchvision 
import transforms
from random import randint
import pickle
from PIL import Image
import numpy as np
from matplotlib.pyplot import imshow
from typing import Dict

import tenseal as ts

In [5]:
def context():
    context = ts.context(ts.SCHEME_TYPE.CKKS, 8192, coeff_mod_bit_sizes=[60, 40, 60, 40])
    context.global_scale = pow(2, 40)
    context.generate_galois_keys()
    return context

context = context()

In [6]:
plain1 = ts.plain_tensor([1,2,3,4], [2,2])

print(" First tensor: Shape = {} Data = {}".format(plain1.shape, plain1.tolist()))

plain2 = ts.plain_tensor(np.array([5,6,7,8]).reshape(2,2))
print(" Second tensor: Shape = {} Data = {}".format(plain2.shape, plain2.tolist()))



 First tensor: Shape = [2, 2] Data = [[1.0, 2.0], [3.0, 4.0]]
 Second tensor: Shape = [2, 2] Data = [[5.0, 6.0], [7.0, 8.0]]


In [7]:
# Encrypted tensor creation
# CKKS requires two operations for encrypting a new message

encrypted_tensor1 = ts.ckks_tensor(context, plain1)
encrypted_tensor2 = ts.ckks_tensor(context, plain2)

print(" Shape = {}".format(encrypted_tensor1.shape))
print(" Encrypted Data = {}.".format(encrypted_tensor1))

encrypted_tensor_from_np = ts.ckks_tensor(context, np.array([5,6,7,8]).reshape([2,2]))
print(" Shape = {}".format(encrypted_tensor_from_np.shape))



 Shape = [2, 2]
 Encrypted Data = <tenseal.tensors.ckkstensor.CKKSTensor object at 0x7fd223e7a828>.
 Shape = [2, 2]


In [8]:
def decrypt(enc):
    return enc.decrypt().tolist()



In [9]:
result = encrypted_tensor1 + encrypted_tensor2
print("Plain equivalent: {} + {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))



Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] + [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[5.99999999954398, 8.000000000125713], [10.000000000090417, 11.999999999339503]].


In [10]:
result = encrypted_tensor1 - encrypted_tensor2
print("Plain equivalent: {} - {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] - [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[-3.9999999998270455, -3.9999999996730726], [-3.999999999535404, -4.000000000117461]].


In [11]:
result = encrypted_tensor1 * encrypted_tensor2
print("Plain equivalent: {} * {}\nDecrypted result: {}.".format(plain1.tolist(), plain2.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] * [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[4.7680844083051965e-06, 1.1443775209771311e-05], [2.0027152801230155e-05, 3.051770893304236e-05]].


In [12]:
plain = ts.plain_tensor([5,6,7,8], [2,2])
result = encrypted_tensor1 * plain

print("Plain equivalent: {} * {}\nDecrypted result: {}.".format(plain1.tolist(), plain.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] * [[5.0, 6.0], [7.0, 8.0]]
Decrypted result: [[4.7683699495556774e-06, 1.1444051837769252e-05], [2.0027862519096242e-05, 3.0517599314438496e-05]].


In [13]:
result = -encrypted_tensor1 

print("Plain equivalent: -{}\nDecrypted result: {}.".format(plain1.tolist(), decrypt(result)))

Plain equivalent: -[[1.0, 2.0], [3.0, 4.0]]
Decrypted result: [[-0.9999999998584672, -2.0000000002263194], [-3.0000000002775065, -3.999999999611022]].


In [14]:

result = encrypted_tensor1 ** 3
print("Plain equivalent: {} ^ 3\nDecrypted result: {}.".format(plain1.tolist(), decrypt(result)))

Plain equivalent: [[1.0, 2.0], [3.0, 4.0]] ^ 3
Decrypted result: [[9.53838909516104e-07, 7.628664477771338e-06], [2.574947783783045e-05, 6.103613227130018e-05]].


In [15]:
result = encrypted_tensor1.polyval([1,0,1,1])

print("X = {}".format(plain1.tolist()))
print("1 + X^2 + X^3 = {}.".format(decrypt(result)))

X = [[1.0, 2.0], [3.0, 4.0]]
1 + X^2 + X^3 = [[1.0000019073975557, 1.00001144287035], [1.0000343322473977, 1.0000762952627855]].


In [16]:
result = encrypted_tensor1.polyval([1,0,1,1])

print("X = {}".format(plain1.tolist()))
print("1 + X^2 + X^3 = {}.".format(decrypt(result)))

X = [[1.0, 2.0], [3.0, 4.0]]
1 + X^2 + X^3 = [[1.0000019073797526, 1.0000114426461435], [1.0000343324803176, 1.0000762953324562]].


In [17]:
# Encrypted inference demo
# We have CKKS scheme 
# classification over the MNIST dataset using a single convolution and two fully connected
# layers with a square activation function. one of the prominent use cases for homomorphic
# encryptin

# Create the TenSEAL security context
def create_ctx():
    """Helper for creating the CKKS context
    CKKS params:
        -Polynomial degree: 8192.
        -Coefficient modulus size: [40,21,21,21,21,21,21,40].
        - Scale: 2 ** 21 .
        - The setup requires the Galois keys for evaluating the convolutions
    """
    poly_mod_degree = 8192
    coeff_mod_bit_sizes = [40,21,21,21,21,21,21,40]
    ctx = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    ctx.global_scale = 2 ** 21
    ctx.generate_galois_keys()
    return ctx
# Sample an image
def load_input():
    transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        
    )
    idx = randint(1, 6)
    img_name = "data/mnist-samples/img_{}.jpg".format(idx)
    print(img_name)
    img = Image.open(img_name)
    return transform(img).view(28, 28).tolist(), img

# Helper for encoding the image
def prepare_input(ctx, plain_input):
    enc_input, windows_nb = ts.im2col_encoding(ctx, plain_input, 7, 7, 3)
    assert windows_nb == 64
    return enc_input

In [20]:
# Server Model
# Load a pretrained model and adapt the forward call for encrypted input
class ConvMNIST():
    """CNN for classifying MNIST data.
    Input should be an encoded 28x28 matrix representing the image.
    TenSEAL can be used for encoding 'tenseal.im2col_encoding(ctx, input_matrix, 7, 7, 3)'
    The input should also be normalized with a mean=0.1307 and an std=0.3081 before encryption"""
    
    def __init__(self, parameters: Dict[str,list]):
        self.conv1_weight = parameters["conv1_weight"]
        self.conv1_bias = parameters["conv1_bias"]
        self.fc1_weight = parameters["fc1_weight"]
        self.fc1_bias = parameters["fc1_bias"]
        self.fc2_weight = parameters["fc2_weight"]
        self.fc2_bias = parameters["fc2_bias"]
        self.windows_nb = parameters["windows_nb"]
    
    def forward(self, enc_x: ts.CKKSVector) -> ts.CKKSVector:
        # conv layer
        channels = []
        for kernel, bias in zip(self.conv1_weight, self.conv1_bias):
            y = enc_x.conv2d_im2col(kernel, self.windows_nb) + bias
            channels.append(y)
        out = ts.CKKSVector.pack_vectors(channels)
        # squaring
        out.square_()
        # no need to flat
        # fc1 layer
        out = out.mm_(self.fc1_weight) + self.fc1_bias
        # squaring
        out.square_()
        # output layer
        out = out.mm_(self.fc2_weight) + self.fc2_bias
        return out

    @staticmethod
    def prepare_input(context: bytes, ckks_vector: bytes) -> ts.CKKSVector:
        try:
            ctx = ts.context_from(context)
            enc_x = ts.ckks_vector_from(ctx, ckks_vector)
        except:
            raise DeserializationError("cannot deserialize context or ckks_vector")
        try:
            _ = ctx.galois_keys()
        except:
            raise InvalidContext("the context doesn't hold galois keys")
        return enc_x
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

In [1]:
#Server helpers
import pickle
import os

def load_parameters(file_path: str) -> dict:
    try:
        parameters = pickle.load(open(file_path, "rb"))
        print(f"Model loaded from '{file_path}'")
    except OSError as ose:
        print("error", ose)
        raise ose
    return parameters



parameters = load_parameters("parameters/ConvMNIST-0.1.pickle")
model = ConvMNIST(parameters)


error [Errno 2] No such file or directory: 'parameters/ConvMNIST-0.1.pickle'


FileNotFoundError: [Errno 2] No such file or directory: 'parameters/ConvMNIST-0.1.pickle'

In [2]:
# Client has to create the CKKS context for the first query. Then, he samples and encrypts
# a random image from the dataset
# The serialized context and encrypted image are sent to the server for evaluation

# CKKS context generation.
context = create_ctx()

# Random image sampling
image, orig = load_input()

# Image encoding
encrypted_image = prepare_input(context, image)

print("Encrypted image ", encrypted_image)
print("Original image ")
imshow(np.asarray(orig))

# We prepare the context for the server, by making it public(we drop the secret key)
server_context = context.copy()
server_context.make_context_public()

# Context and ciphertext serialization
server_context = server_context.serialize()
encrypted_image = encrypted_image.serialize()


client_query = {
    "data" : encrypted_image,
    "context" : server_context,
}

NameError: name 'create_ctx' is not defined

In [3]:
encrypted_query = model.prepare

NameError: name 'model' is not defined